In [159]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import re
import string
import nltk
nltk.download('stopwords')

from keras import backend as K
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow import keras
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn import svm, metrics
from sklearn import preprocessing

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Spam detection

In [2]:
db_names = ["word_freq_make", "word_freq_address", "word_freq_all", "word_freq_3d", "word_freq_our", "word_freq_over", 
         "word_freq_remove", "word_freq_internet", "word_freq_order", "word_freq_mail", "word_freq_receive", "word_freq_will", 
         "word_freq_people", "word_freq_report", "word_freq_addresses", "word_freq_free", "word_freq_business", "word_freq_email",
         "word_freq_you", "word_freq_credit", "word_freq_your", "word_freq_font", "word_freq_000", "word_freq_money",
         "word_freq_hp", "word_freq_hpl", "word_freq_george", "word_freq_650", "word_freq_lab", "word_freq_labs",
         "word_freq_telnet", "word_freq_857", "word_freq_data", "word_freq_415", "word_freq_85", "word_freq_technology",
         "word_freq_1999", "word_freq_parts", "word_freq_pm", "word_freq_direct", "word_freq_cs", "word_freq_meeting", 
         "word_freq_original", "word_freq_project", "word_freq_re", "word_freq_edu", "word_freq_table", "word_freq_conference",
         "char_freq_;", "char_freq_(", "char_freq_[", "char_freq_!", "char_freq_$", "char_freq_#", "capital_run_length_average",
         "capital_run_length_longest", "capital_run_length_total", "class"]

In [75]:
df = pd.read_csv("spambase_lab_csv.csv")
df.columns = db_names
df.head()

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,class
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1


In [78]:
features = df.iloc[:,0:-1]
labels = df.iloc[:,-1]

In [79]:
features

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,word_freq_conference,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.0,0.000,0.000,0.0,0.778,0.000,0.000,3.756,61,278
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.0,0.000,0.132,0.0,0.372,0.180,0.048,5.114,101,1028
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.0,0.010,0.143,0.0,0.276,0.184,0.010,9.821,485,2259
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.0,0.000,0.137,0.0,0.137,0.000,0.000,3.537,40,191
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.0,0.000,0.135,0.0,0.135,0.000,0.000,3.537,40,191
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4596,0.31,0.00,0.62,0.0,0.00,0.31,0.00,0.00,0.00,0.00,...,0.0,0.000,0.232,0.0,0.000,0.000,0.000,1.142,3,88
4597,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,0.000,0.000,0.0,0.353,0.000,0.000,1.555,4,14
4598,0.30,0.00,0.30,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,0.102,0.718,0.0,0.000,0.000,0.000,1.404,6,118
4599,0.96,0.00,0.00,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.0,0.000,0.057,0.0,0.000,0.000,0.000,1.147,5,78


In [80]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size = 0.25, random_state = 0)

In [81]:
model = keras.Sequential()
model.add(Dense(512, activation='relu', input_shape=(57,)))
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.build()

In [82]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

In [83]:
precision = tf.keras.metrics.Precision()
recall = tf.keras.metrics.Recall()

In [84]:
model.compile(
  optimizer='adam',
  loss='binary_crossentropy',
  metrics=['accuracy', precision, recall],
)

In [86]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_33 (Dense)            (None, 512)               29696     
                                                                 
 dense_34 (Dense)            (None, 512)               262656    
                                                                 
 dense_35 (Dense)            (None, 1)                 513       
                                                                 
Total params: 292,865
Trainable params: 292,865
Non-trainable params: 0
_________________________________________________________________


In [87]:
history = model.fit(
  X_train,
  y_train, 
  epochs=50,
  batch_size=16,
  callbacks = [callback]
)

Epoch 1/50
216/216 [==============================] - 1s 2ms/step - loss: 2.8141 - accuracy: 0.7055 - precision_2: 0.6052 - recall_2: 0.7162
Epoch 2/50
216/216 [==============================] - 0s 2ms/step - loss: 1.4309 - accuracy: 0.8162 - precision_2: 0.7441 - recall_2: 0.8101
Epoch 3/50
216/216 [==============================] - 0s 2ms/step - loss: 1.2512 - accuracy: 0.8339 - precision_2: 0.7668 - recall_2: 0.8285
Epoch 4/50
216/216 [==============================] - 0s 2ms/step - loss: 0.5305 - accuracy: 0.8786 - precision_2: 0.8374 - recall_2: 0.8566
Epoch 5/50
216/216 [==============================] - 0s 2ms/step - loss: 0.7791 - accuracy: 0.8635 - precision_2: 0.8050 - recall_2: 0.8603
Epoch 6/50
216/216 [==============================] - 0s 2ms/step - loss: 0.6362 - accuracy: 0.8768 - precision_2: 0.8295 - recall_2: 0.8633
Epoch 7/50
216/216 [==============================] - 0s 2ms/step - loss: 0.3980 - accuracy: 0.9104 - precision_2: 0.8783 - recall_2: 0.8958
Epoch 8/50
21

In [88]:
model.evaluate(
  X_test,
  y_test
)

36/36 [==============================] - 0s 2ms/step - loss: 0.4687 - accuracy: 0.8836 - precision_2: 0.8260 - recall_2: 0.8978


[0.4686857759952545, 0.8835794925689697, 0.8259999752044678, 0.897826075553894]

# Класифікація зображень

In [105]:
train_dir = "C:/Users/User/Downloads/archive (1)/Chessman-image-dataset/Chess"
train_image_generator = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_data_gen = train_image_generator.flow_from_directory(directory=train_dir, batch_size=32, shuffle=True, class_mode='categorical', seed=42,
                                                          subset='training')

valid_data_gen = train_image_generator.flow_from_directory(directory=train_dir, batch_size=32, shuffle=True, class_mode='categorical', seed=42,
                                                          subset='validation')

Found 442 images belonging to 6 classes.
Found 109 images belonging to 6 classes.


In [110]:
number_of_classes = 6
img_rows, img_cols = 256, 256 
input_shape = (img_rows, img_cols,3)

In [132]:
model = keras.Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(number_of_classes, activation='softmax'))

In [133]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=(['accuracy'])
             )

model.summary()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_17 (Conv2D)          (None, 254, 254, 32)      896       
                                                                 
 conv2d_18 (Conv2D)          (None, 252, 252, 64)      18496     
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 126, 126, 64)     0         
 2D)                                                             
                                                                 
 dropout_14 (Dropout)        (None, 126, 126, 64)      0         
                                                                 
 flatten_7 (Flatten)         (None, 1016064)           0         
                                                                 
 dense_49 (Dense)            (None, 128)               130056320 
                                                     

In [134]:
history = model.fit(train_data_gen,
          batch_size=64,
          epochs=20,
          callbacks = [callback],
          validation_data=valid_data_gen)

Epoch 1/20
14/14 [==============================] - 42s 3s/step - loss: 25.6945 - accuracy: 0.1652 - val_loss: 1.7810 - val_accuracy: 0.2752
Epoch 2/20
14/14 [==============================] - 40s 3s/step - loss: 1.7561 - accuracy: 0.2828 - val_loss: 1.8086 - val_accuracy: 0.2752
Epoch 3/20
14/14 [==============================] - 40s 3s/step - loss: 1.5941 - accuracy: 0.3891 - val_loss: 1.7791 - val_accuracy: 0.3119
Epoch 4/20
14/14 [==============================] - 41s 3s/step - loss: 1.1646 - accuracy: 0.6267 - val_loss: 1.7387 - val_accuracy: 0.3119
Epoch 5/20
14/14 [==============================] - 40s 3s/step - loss: 0.8039 - accuracy: 0.7783 - val_loss: 1.9013 - val_accuracy: 0.3303
Epoch 6/20
14/14 [==============================] - 40s 3s/step - loss: 0.5180 - accuracy: 0.8597 - val_loss: 2.1186 - val_accuracy: 0.3670
Epoch 7/20
14/14 [==============================] - 41s 3s/step - loss: 0.4038 - accuracy: 0.8959 - val_loss: 3.0427 - val_accuracy: 0.3670
Epoch 8/20
14/14 [=

# Класифікація тексту

In [141]:
df_f = pd.read_csv("Fake.csv")
df_f["label"] = 0
df_t = pd.read_csv("True.csv")
df_t["label"] = 1
df = pd.concat([df_f, df_t], ignore_index=True)

In [143]:
df.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [145]:
df_text = df.text

In [147]:
df_text[0]

'Donald Trump just couldn t wish all Americans a Happy New Year and leave it at that. Instead, he had to give a shout out to his enemies, haters and  the very dishonest fake news media.  The former reality show star had just one job to do and he couldn t do it. As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year,  President Angry Pants tweeted.  2018 will be a great year for America! As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year. 2018 will be a great year for America!  Donald J. Trump (@realDonaldTrump) December 31, 2017Trump s tweet went down about as welll as you d expect.What kind of president sends a New Year s greeting like this despicable, petty, infantile gibberish? Only Trump! His lack of decency won t ev

In [149]:
df_text = df_text.str.replace("\n"," ").str.replace("\r"," ")

In [154]:
translator = re.compile('[%s]' % re.escape(string.punctuation))
df_text = df_text.apply(lambda x: translator.sub(' ', x))

In [155]:
df_text = df_text.str.lower()

In [156]:
df_text[0]

'donald trump just couldn t wish all americans a happy new year and leave it at that  instead  he had to give a shout out to his enemies  haters and  the very dishonest fake news media   the former reality show star had just one job to do and he couldn t do it  as our country rapidly grows stronger and smarter  i want to wish all of my friends  supporters  enemies  haters  and even the very dishonest fake news media  a happy and healthy new year   president angry pants tweeted   2018 will be a great year for america  as our country rapidly grows stronger and smarter  i want to wish all of my friends  supporters  enemies  haters  and even the very dishonest fake news media  a happy and healthy new year  2018 will be a great year for america   donald j  trump   realdonaldtrump  december 31  2017trump s tweet went down about as welll as you d expect what kind of president sends a new year s greeting like this despicable  petty  infantile gibberish  only trump  his lack of decency won t ev

In [160]:
stop_words = stopwords.words('english')
df_text = df_text.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

In [161]:
df_text[0]

'donald trump wish americans happy new year leave instead give shout enemies haters dishonest fake news media former reality show star one job country rapidly grows stronger smarter want wish friends supporters enemies haters even dishonest fake news media happy healthy new year president angry pants tweeted 2018 great year america country rapidly grows stronger smarter want wish friends supporters enemies haters even dishonest fake news media happy healthy new year 2018 great year america donald j trump realdonaldtrump december 31 2017trump tweet went welll expect kind president sends new year greeting like despicable petty infantile gibberish trump lack decency even allow rise gutter long enough wish american citizens happy new year bishop talbert swan talbertswan december 31 2017no one likes calvin calvinstowell december 31 2017your impeachment would make 2018 great year america also accept regaining control congress miranda yaver mirandayaver december 31 2017do hear talk include ma

In [173]:
tokenizer = keras.preprocessing.text.Tokenizer(10000)
tokenizer.fit_on_texts(df_text)
df_text_sequence = tokenizer.texts_to_sequences(df_text)

In [174]:
X_train, X_test, y_train, y_test = train_test_split(df_text_sequence, df.label, test_size = 0.25, random_state = 0)

In [175]:
max_seq_len = 100

In [176]:
X_train_pad = keras.preprocessing.sequence.pad_sequences(X_train, maxlen=max_seq_len)
X_test_pad = keras.preprocessing.sequence.pad_sequences(X_test, maxlen=max_seq_len)

In [177]:
X_train_pad[0]

array([   0,    0,    0,    0,    0,    0, 1507,   65, 2128,  706, 2662,
       1027, 2857,    9, 4928, 3910,  643,  830, 4482,  232,   59, 3629,
        274,   70,    9, 1057,  997,  117, 3910, 4067, 5355,  156,  830,
       5795, 6008, 8689, 2704, 4810, 1507,   95,  286,   44,  524, 1421,
        465,   59, 1507, 7533,  173,   98, 1272,  830, 5795,  758,  324,
        295,  573,  706,   69, 1348,  861,    9,  226, 2821,  187, 1027,
       4653,  232,   21,  985, 6823,  748, 6008, 1827, 1507,    9, 1057,
       2995, 3270,   15,   69,   68, 3629, 2350, 7460,  574,  136, 2691,
          6, 1507,    3, 5552, 1031, 4032,   96,  279,  141, 1031,  377,
        229])

In [184]:
emb_dim = 128

model = keras.Sequential()
model.add(layers.Embedding(10000, emb_dim))
model.add(layers.LSTM(128))
model.add(layers.Dense(1, activation='sigmoid'))

In [185]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=(['accuracy'])
             )

model.summary()

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 128)         1280000   
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dense_52 (Dense)            (None, 1)                 129       
                                                                 
Total params: 1,411,713
Trainable params: 1,411,713
Non-trainable params: 0
_________________________________________________________________


In [186]:
history = model.fit(X_train_pad,
          y_train,   
          batch_size=512,
          epochs=20,
          callbacks = [callback])

Epoch 1/20
66/66 [==============================] - 40s 584ms/step - loss: 0.2375 - accuracy: 0.8825
Epoch 2/20
66/66 [==============================] - 39s 598ms/step - loss: 0.0264 - accuracy: 0.9926
Epoch 3/20
66/66 [==============================] - 39s 596ms/step - loss: 0.0083 - accuracy: 0.9985
Epoch 4/20
66/66 [==============================] - 39s 591ms/step - loss: 0.0033 - accuracy: 0.9994
Epoch 5/20
66/66 [==============================] - 39s 583ms/step - loss: 0.0017 - accuracy: 0.9998
Epoch 6/20
66/66 [==============================] - 38s 580ms/step - loss: 0.0010 - accuracy: 0.9999
Epoch 7/20
66/66 [==============================] - 38s 579ms/step - loss: 0.0011 - accuracy: 0.9999
Epoch 8/20
66/66 [==============================] - 38s 584ms/step - loss: 0.0045 - accuracy: 0.9989
Epoch 9/20
66/66 [==============================] - 41s 622ms/step - loss: 0.0022 - accuracy: 0.9996
